In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import aplpy
from math import ceil
from astropy import stats 
from glob import glob
import matplotlib as mpl
from tqdm import tqdm
import colorcet
import multicolorfits as mcf
from PIL import Image
import os
import imageio
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/Users/abarnes/GitHub/misc/')
from data_cube_analysis import getdata

plt.style.use('paper.mplstyle')

multicolorfits: kapteyn package required for some options in task reproject2D()


In [2]:
def get_channel_maps(input, start=-200, stop=200, step=20, method='mom0'):
    """
    This function calculates the zeroth moment (total intensity) for each velocity channel in a data cube,
    and writes the results to a FITS file.

    Args:
    input (str, cube): Filename or cube object... 
    start (int): The start of the velocity range for channel extraction. Default is -200 km/s.
    stop (int): The end of the velocity range for channel extraction. Default is 200 km/s.
    step (int): The velocity step size for channel extraction. Default is 20 km/s.

    Returns:
    None
    """

    # Use the get_cube function from the getdata module to extract the data cube from the FITS file.
    cube = getdata.get_cube(input)
    # if isinstance(input, str):
    #     cube = getdata.get_cube(input)
    # else: 
    #     cube = input
    
    # Define the velocity channels.
    channels = np.arange(start, stop, step) *u.km/u.s

    # Get the number of channels.
    n = len(channels)-1

    # Initialize an array to hold the channel map data.
    channelmaps_data = ['']*(n)

    # For each channel...
    for i in tqdm(range(n)): 

        # ...extract the cube for that channel...
        cube_channels = cube.spectral_slab(channels[i], channels[i+1])

        # ...and calculate the zeroth moment, storing the result in the array.
        if method=='mom0':
            channelmaps_data[i] = cube_channels.moment0().hdu.data
        elif method=='max':
            channelmaps_data[i] = cube_channels.max(axis=(0))
        channelmaps_data[i][channelmaps_data[i] == 0] = np.nan

        # Get the FITS header from the last cube's zeroth moment.
        channelmaps_header = cube_channels.moment0().hdu.header

        # if method=='mom0':
        #     channelmaps_header = cube_channels.moment0().hdu.header
        # elif method=='max':
        #     channelmaps_header = cube_channels.max(axis=(0,1))

    # Create a FITS HDU using the channel map data and the header.
    channelmaps_hdu = fits.PrimaryHDU(channelmaps_data, channelmaps_header)

    # Define the output file name by removing '.fits' from the input file name.
    outputfile = input.replace('.fits', '')
 
    # Write the HDU to a FITS file.
    channelmaps_hdu.writeto('%s_chans.fits' %outputfile, overwrite=True)

    return(channelmaps_hdu)

In [3]:
def plot_channel_maps_indv(channelmaps_hdu, fileappend='', outputdir='../figures/', 
                           start=-200, stop=200, step=20, 
                           vmin=None, vmax=None, 
                           l=0.77, b=-0.178, width=0.265, height=0.225, 
                           show_bubble=False, figsize=(5,5), dpi=150, 
                           velospread=5):
    """
    This function plots all the channel maps contained in a FITS HDU in a grid with a specified number of columns.
    It shows the velocity range in the upper right corner of each plot.

    Args:
    channelmaps_hdu (HDU): The FITS HDU containing the channel maps.
    start (int): The start of the velocity range for channel extraction. Default is -200 km/s.
    stop (int): The end of the velocity range for channel extraction. Default is 200 km/s.
    step (int): The velocity step size for channel extraction. Default is 20 km/s.

    Returns:
    None
    """
    # Determine the number of channels from the FITS HDU.
    channels = channelmaps_hdu.header['NAXIS3']
    
    # For each channel...
    n = channels-2
    for chan in tqdm(range(n)):
        
        # if chan!=0:
        #     continue

        # ...create a copy of the FITS HDU...
        channelmaps_hdu_ = channelmaps_hdu.copy()
        
        # ...replace the data in the HDU with the data for the current channel...
        channelmaps_hdu_.data = channelmaps_hdu_.data[chan]
        
        # Make RGB
        data = channelmaps_hdu.data

        if chan < velospread: 
            chans0 = list(np.arange(0, chan))
            if len(chans0)==0:
                chans0 = [chan]
        else: 
            chans0 = list(np.arange(chan-velospread, chan))

        if chan > n-velospread: 
            chans1 = list(np.arange(chan+1, n))
            if len(chans1)==0:
                chans1 = [chan]
        else: 
            chans1 = list(np.arange(chan+1, chan+1+velospread))

        # print(chans0, chan, chans1)

        # c0 = np.nanmax(data[chans0], axis=(0))
        # c1 = data[chan]
        # c2 = np.nanmax(data[chans1], axis=(0))

        c0 = data[np.nanmin(chans0)]
        c1 = data[chan]
        c2 = data[np.nanmax(chans1)]

        c0_grey = mcf.greyRGBize_image(c0, rescalefn='linear', min_max=[vmin, vmax]) 
        c1_grey = mcf.greyRGBize_image(c1, rescalefn='linear', min_max=[vmin, vmax]) 
        c2_grey = mcf.greyRGBize_image(c2, rescalefn='linear', min_max=[vmin, vmax])

        c0_r = mcf.colorize_image(c0_grey, '#FF0000', colorintype='hex')
        c1_g = mcf.colorize_image(c1_grey, '#00FF00', colorintype='hex') 
        c2_b = mcf.colorize_image(c2_grey, '#0000FF', colorintype='hex')

        # c0_r = mcf.colorize_image(c0_grey, '#994242', colorintype='hex')
        # c1_g = mcf.colorize_image(c1_grey, '#FFF9DB', colorintype='hex') 
        # c2_b = mcf.colorize_image(c2_grey, '#1773E9', colorintype='hex')

        comb_rgb = mcf.combine_multicolor([c0_r, c1_g, c2_b])
        comb_rgb_s = (comb_rgb*255).astype(np.uint8)

        # Save png
        alpha = Image.fromarray(comb_rgb_s[::-1], mode="RGB")
        alpha.save('tmp.jpeg', format='jpeg')

        # Make plot
        fig = plt.figure(figsize=figsize)

        gc = aplpy.FITSFigure(channelmaps_hdu_, subplot=(1, 1, 1), figure=fig)  
        gc.recenter(l, b, width=width, height=height)

        gc.show_rgb('tmp.jpeg')

        gc.show_contour(fits.PrimaryHDU(np.isnan(channelmaps_hdu_.data)*1, channelmaps_hdu_.header), levels=[1], linewidths=1, colors='white')

        # Remove ticks etc
        gc.axis_labels.hide()
        gc.tick_labels.hide()
        gc.ticks.set_color('black')
        gc.ticks.set_linewidth(2)
        gc.set_nan_color('none')

        # ...and display the velocity range in the upper right corner.
        step1 = start + (chan)*step
        # step2 = start + (chan)*step
        # velo_range = "%0.1f to %0.1f km/s" %(step1, step2)
        velo_range = "%0.1f km/s" %(step1)
        gc.add_label(0.05, 0.95, velo_range, relative=True, color='black', bbox=dict(facecolor='white', boxstyle='round', alpha=0.75), ha='left')

        if show_bubble:

            xcentre = 0.8041918903
            ycentre = -0.1838483592
            r_inner = 79/3600
            r_outer = 154/3600
            gc.show_ellipses(xcentre, ycentre, r_inner*2, r_inner*2, edgecolor='white', linestyles='dotted', linewidths=2, zorder=10., alpha=0.9)
            gc.show_ellipses(xcentre, ycentre, r_outer*2, r_outer*2, edgecolor='white', linestyles='dotted', linewidths=2, zorder=10., alpha=0.9)

        # Adjust the subplot layout to reduce space between subplots
        fig.tight_layout(h_pad=0, w_pad=-0.6)
        plt.subplots_adjust(wspace=-0.06, hspace=0)
        
        # Save the figure. 
        fig.savefig(outputdir+fileappend+'_%i.jpeg' %chan, bbox_inches='tight', dpi=dpi)
    
        plt.close('all')
        
        os.system('rm tmp.jpeg')

    return()

In [7]:
ls /Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma_jun2024/cubes/

ACES_TP_spw17_mosaic.fits*
ACES_TP_spw19_mosaic.fits*
ACES_TP_spw21_mosaic.fits*
ACES_TP_spw23_mosaic.fits*
ACES_TP_spw25_mosaic.fits*
ACES_TP_spw27_mosaic.fits*
CS21_CubeMosaic_downsampled9.fits*
CS21_CubeMosaic_downsampled9_spectrally.fits*
H13CN_CubeMosaic_downsampled9.fits*
H13COp_CubeMosaic_downsampled9.fits*
H13COp_CubeMosaic_downsampled9_spectrally.fits*
HC3N_CubeMosaic_downsampled9.fits*
HC3N_CubeMosaic_downsampled9_spectrally.fits*
HN13C_CubeMosaic_downsampled9.fits*
HN13C_CubeMosaic_downsampled9_spectrally.fits*
HNCO_7mTP_CubeMosaic.fits*
HNCO_7mTP_CubeMosaic_downsampled9.fits*
HNCO_7mTP_CubeMosaic_downsampled9_spectrally.fits*
SO32_CubeMosaic_downsampled9.fits*
SO32_CubeMosaic_downsampled9_spectrally.fits*
SiO21_CubeMosaic_downsampled9.fits*
SiO21_CubeMosaic_downsampled9_spectrally.fits*


In [10]:
inputfile = '/Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma_jun2024/cubes/SiO21_CubeMosaic_downsampled9.fits'
print(inputfile)
hdu = fits.open(inputfile)[0]
print(hdu)

line = 'sio'
start=-150
stop=150
step=1

# start=10
# stop=20
# step=0.5

channelmaps_hdu = get_channel_maps(inputfile, 
                                   start=start, 
                                   stop=stop, 
                                   step=step, 
                                   method='max')

channelmaps_hdu_f32 = channelmaps_hdu.copy()
channelmaps_hdu_f32.data = np.float32(channelmaps_hdu.data)

# ensuring plotting over same scale
data = channelmaps_hdu_f32.data.copy()
vmin, vmax = np.nanpercentile(data, [1, 99.9])

plot_channel_maps_indv(channelmaps_hdu_f32, 
                start=start, 
                stop=stop, 
                step=step, 
                vmin = vmin, 
                vmax = vmax, 
                l=0.1404297, b=-0.0206618, width=1.5116547, height=0.5356110,
                figsize=(10,10),
                velospread=10,
                fileappend = '%s_chans_wholecmz_rgb/sio_chans_wholecmz_rgb' %line)

filenames = glob('../figures/%s_chans_wholecmz_rgb/*.jpeg' %line)
filenames_sorted = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))
images = []
for filename in filenames_sorted:
    images.append(imageio.imread(filename))
imageio.mimsave('../figures/%s_chans_wholecmz_rgb/%s_chans_wholecmz_rgb_movie.gif' %line, images)

100%|██████████| 297/297 [07:59<00:00,  1.61s/it]


In [12]:
inputfile = '/Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma_jun2024/cubes/H13COp_CubeMosaic_downsampled9.fits'
print(inputfile)
hdu = fits.open(inputfile)[0]
print(hdu)

line = 'h13cop'
start=-150
stop=150
step=1

# start=10
# stop=20
# step=0.5

channelmaps_hdu = get_channel_maps(inputfile, 
                                   start=start, 
                                   stop=stop, 
                                   step=step, 
                                   method='max')

channelmaps_hdu_f32 = channelmaps_hdu.copy()
channelmaps_hdu_f32.data = np.float32(channelmaps_hdu.data)

# ensuring plotting over same scale
data = channelmaps_hdu_f32.data.copy()
vmin, vmax = np.nanpercentile(data, [1, 99.9])

plot_channel_maps_indv(channelmaps_hdu_f32, 
                start=start, 
                stop=stop, 
                step=step, 
                vmin = vmin, 
                vmax = vmax, 
                l=0.1404297, b=-0.0206618, width=1.5116547, height=0.5356110,
                figsize=(10,10),
                velospread=10,
                fileappend = '%s_chans_wholecmz_rgb/sio_chans_wholecmz_rgb' %line)

filenames = glob('../figures/%s_chans_wholecmz_rgb/*.jpeg' %line)
filenames_sorted = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))
images = []
for filename in filenames_sorted:
    images.append(imageio.imread(filename))
imageio.mimsave('../figures/%s_chans_wholecmz_rgb/%s_chans_wholecmz_rgb_movie.gif' %(line,line), images)

In [ ]:
inputfile = '/Users/abarnes/Dropbox/work/Smallprojects/aces/data/alma_jun2024/cubes/H13COp_CubeMosaic_downsampled9.fits'
print(inputfile)
hdu = fits.open(inputfile)[0]
print(hdu)

line = 'h13cop'
start=-150
stop=150
step=1

# start=10
# stop=20
# step=0.5

channelmaps_hdu = get_channel_maps(inputfile, 
                                   start=start, 
                                   stop=stop, 
                                   step=step, 
                                   method='max')

channelmaps_hdu_f32 = channelmaps_hdu.copy()
channelmaps_hdu_f32.data = np.float32(channelmaps_hdu.data)

# ensuring plotting over same scale
data = channelmaps_hdu_f32.data.copy()
vmin, vmax = np.nanpercentile(data, [1, 99.9])

plot_channel_maps_indv(channelmaps_hdu_f32, 
                start=start, 
                stop=stop, 
                step=step, 
                vmin = vmin, 
                vmax = vmax, 
                l=0.1404297, b=-0.0206618, width=1.5116547, height=0.5356110,
                figsize=(10,10),
                velospread=10,
                fileappend = '%s_chans_wholecmz_rgb/sio_chans_wholecmz_rgb' %line)

filenames = glob('../figures/%s_chans_wholecmz_rgb/*.jpeg' %line)
filenames_sorted = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))
images = []
for filename in filenames_sorted:
    images.append(imageio.imread(filename))
imageio.mimsave('../figures/%s_chans_wholecmz_rgb/%s_chans_wholecmz_rgb_movie.gif' %line, images)